In [79]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd
from datetime import timedelta

page = requests.get("http://www.90minut.pl/liga/1/liga12330.html")
soup = BeautifulSoup(page.content, features="html.parser")
scores = soup.findAll("a", {"class": "main"})
dates = soup.findAll("td", valign="top", align="left",width="190")
soup = soup.findAll("td", valign="top", width="180")
months = {
    "stycznia": "January",
    "lutego": "February",
    "marca": "March",
    "kwietnia": "April",
    "maja": "May",
    "czerwca": "June",
    "lipca": "July",
    "sierpnia": "August",
    "września": "September",
    "października": "October",
    "listopada": "November",
    "grudnia": "December"
}

In [80]:
def delete_comma(text:str)->str:
    if ',' in text:
        return text.split(',')[0]
    else:
        return text

In [81]:
def pl_months_to_eng(data_str:str)->datetime:


    day, month, year = data_str.split()


    month = months[month]


    data_str_en = f"{day} {month} {year}"


    data = datetime.strptime(data_str_en, "%d %B %Y")
    return data

In [82]:
def add_points(team:str,points:float,table):
       table[table(table["Team_name"].equals(team)).index,1]=+points

In [83]:
EXTRA_POINTS_WIN=0.25
EXTRA_POINTS_DRAW=0.0
POINTS_WIN=3.0
POINTS_DRAW=1.0
POINTS_LOST=0.0
pd.options.mode.chained_assignment = None  # default='warn'

In [84]:
team = []
score = []
date=[]
new_date=[]
for i in soup:
    team.append(i.getText())
for i in scores:
    score.append(i.getText())
for i in dates:
    date.append(i.getText())

score=score[:len(score)-2]
score=score[-307:]
score=score[:len(score)]
score.remove('(wo) ')
new_date=list(map(delete_comma,date))

home=[]
away=[]
for i in range(0,len(team)):
    if i%2==0:
        home.append(team[i])
    else:
        away.append(team[i])   
for i in range(0,len(new_date)):
    if "lipca" in new_date[i] or "sierpnia" in new_date[i] or "września" in new_date[i] or "października" in new_date[i] or "listopada" in new_date[i]:
        new_date[i]+=" 2021"
    else:
        new_date[i]+= " 2022"
        

    

In [85]:
t_date=list(map(pl_months_to_eng,new_date))


In [86]:
home_goal=[]
away_goal=[]
for i in score:
    home_goal.append(i[0])
    away_goal.append(i[2])



In [87]:
dataMatrix={'Home_team':home,
            'Home_score':list(map(int,home_goal)),
            'Away_score':list(map(int,away_goal)),
            'Away_team':away,
            'date':t_date}
out_data=pd.DataFrame(dataMatrix)


In [88]:
points_home=[]
points_away=[]
for i in range(0,len(out_data.iloc[:,0])):
    if (out_data.iloc[i,1]>out_data.iloc[i,2]):
        points_home.append(POINTS_WIN)
        points_away.append(POINTS_LOST)
    elif (out_data.iloc[i,1]<out_data.iloc[i,2]):
        points_home.append(POINTS_LOST)
        points_away.append(POINTS_WIN)
    else:
        points_home.append(POINTS_DRAW)
        points_away.append(POINTS_DRAW)
out_data["Points_home"]=points_home
out_data["Points_away"]=points_away       

In [89]:
team=out_data["Home_team"].unique()
points=[0]*len(team)
Table={"Team_name":team,
      "Points":points}
Table=pd.DataFrame(Table)



In [90]:


for i in range(0,len(out_data["Home_team"])):    
    p=out_data.iloc[i]["Points_home"]
    t=out_data.iloc[i]["Home_team"]
    for j in range(0,len(Table['Team_name'])):
        if Table.iloc[j]['Team_name'] == t:
            Table.iat[j,1]=Table.iat[j,1]+p       
    p=out_data.iloc[i]["Points_away"]
    t=out_data.iloc[i]["Away_team"]
    for j in range(0,len(Table['Team_name'])):
        if Table.iloc[j]['Team_name'] == t:
            Table.iat[j,1]=Table.iat[j,1]+p      
    



In [91]:
out_data=out_data.sort_values(by=['date'], ascending=True)
print(out_data)

                     Home_team  Home_score  Away_score           Away_team  \
6          Raków Częstochowa             1           0      Warta Poznań     
8             Zagłębie Lubin             0           0     Śląsk Wrocław     
1      Jagiellonia Białystok             2           0     Piast Gliwice     
2              Korona Kielce             1           1    Legia Warszawa     
3                Lech Poznań             0           2       Stal Mielec     
..                         ...         ...         ...                 ...   
300            Miedź Legnica             0           0     Górnik Zabrze     
301            Piast Gliwice             3           0     Lechia Gdańsk     
302           Pogoń Szczecin             4           0    Radomiak Radom     
303        Raków Częstochowa             1           1    Zagłębie Lubin     
305              Widzew Łódź             0           3     Korona Kielce     

          date  Points_home  Points_away  
6   2021-07-15      

In [92]:
points_home=[]
points_away=[]
for i in range(0,len(out_data.iloc[:,0])):
    if (out_data.iloc[i,1]>out_data.iloc[i,2]):
        points_home.append(EXTRA_POINTS_WIN)
        points_away.append(-EXTRA_POINTS_WIN)
    elif (out_data.iloc[i,1]<out_data.iloc[i,2]):
        points_home.append(-EXTRA_POINTS_WIN)
        points_away.append(EXTRA_POINTS_WIN)
    else:
        points_home.append(EXTRA_POINTS_DRAW)
        points_away.append(EXTRA_POINTS_DRAW)
out_data["Home_team_temp_points"]=points_home
out_data["Away_team_temp_points"]=points_away

In [93]:

def get_team_form(team_name:str)->dict:
    tmpTeam=(out_data[out_data["Away_team"].str.contains(team_name) | out_data["Home_team"].str.contains(team_name)])
    tmpTeam=tmpTeam.sort_values(by=["date"],ascending=True)
    dateDiff=[]
    for i in range(1,len(tmpTeam.iloc[:,0])):
        dateDiff.append((tmpTeam.iloc[i][4]-tmpTeam.iloc[i-1][4])/(60*60*24))
    dateDiff.insert(0,0)

    tmpTeam["Data_difference"]=dateDiff


    tmpTeamPoints=[]
    for i in range (0,len(tmpTeam.iloc[:,0])):
        if(tmpTeam.iloc[i][0]==team_name):
            tmpTeamPoints.append(tmpTeam.iloc[i][7])
        if(tmpTeam.iloc[i][3]==team_name):
            tmpTeamPoints.append(tmpTeam.iloc[i][8])
                    



    dateDiff=[i.total_seconds() for i in dateDiff if isinstance(i,timedelta)]

    dateDiff.insert(0,0)
    
    addedPoints=[]
    j=0
    for i in range(len(tmpTeamPoints)): 
        if i<4:
            addedPoints.append(tmpTeamPoints[i])
        else:
            if (dateDiff[i]==max(dateDiff) and j==0) or (j<4 and j>0):
                j+=1
                addedPoints.append(tmpTeamPoints[i])
            else:    
                addedPoints.append(round(sum(tmpTeamPoints[i-4:i]),2))

    p_dict = dict(zip(tmpTeam["Index"], addedPoints))
    d_dict = dict(zip(tmpTeam["Index"], dateDiff))
    return p_dict,d_dict


In [94]:

out_data["Index"]=out_data.index

out_data=out_data.assign(Accumulated_home_points=None)
out_data=out_data.assign(Accumulated_away_points=None)
out_data=out_data.assign(Add_Accumulated_home_points=None)
out_data=out_data.assign(Add_Accumulated_away_points=None)
out_data=out_data.sort_values(by=["Index"],ascending=True)

In [95]:


for t in team:
    dic,dateDif=get_team_form(t)
    
    for key in dic:
        for i in range(0,out_data.shape[0]):
            if key==i and t in out_data.iloc[i][0]:
                out_data.iat[i,10]=dic[key]
                out_data.iat[i,12]=dateDif[key]
            if key==i and t in out_data.iloc[i][3]:
                out_data.iat[i,11]=dic[key]
                out_data.iat[i,13]=dateDif[key]    

In [96]:

out_data['Add_Accumulated_home_points'] = pd.to_numeric(out_data['Add_Accumulated_home_points'], errors='coerce').astype(float)
out_data['Add_Accumulated_away_points'] = pd.to_numeric(out_data['Add_Accumulated_away_points'], errors='coerce').astype(float)

largest_values=[]
largest_values.append(out_data["Add_Accumulated_home_points"].nlargest(9))
largest_values.append(out_data["Add_Accumulated_away_points"].nlargest(9))



In [97]:
out_data=out_data.assign(Should_add_accumulated_points_home=None)
out_data=out_data.assign(Should_add_accumulated_points_away=None)
out_data=out_data.sort_values(by=["date"],ascending=True)
for i in range(0,out_data.shape[0]):
    if i in range(0,9):
        out_data.iat[i,14],out_data.iat[i,15]=1,1
    if (out_data.iat[i,12] == largest_values).any():
         out_data.iat[i,14]=1
    if (out_data.iat[i,13] == largest_values).any():
         out_data.iat[i,15]=1
    


In [98]:
for team_name in team:
    modified_rows=(out_data[out_data["Away_team"].str.contains(team_name) | out_data["Home_team"].str.contains(team_name)])
    indexes=[]
    for i in range(0,modified_rows.shape[0]):
        if modified_rows.iat[i,14]==1 and modified_rows.iat[i,0]==team_name:
            indexes+=list(range(i,i+4))
            if i+4>modified_rows.shape[0]:
                break
            else:
                i=i+4
        if modified_rows.iat[i,15]==1 and modified_rows.iat[i,3]==team_name:
            indexes+=list(range(i,i+4))
            if i+4>modified_rows.shape[0]:
                break    
            else:
                i=i+4
    for i in range(0,modified_rows.shape[0]):
        if modified_rows.iat[i,0]==team_name and i in indexes:
            modified_rows.iat[i,14]=1
        elif modified_rows.iat[i,3]==team_name and i in indexes:
            modified_rows.iat[i,15]=1               
    out_data.update(modified_rows)

In [99]:
out_data=out_data.assign(Final_accumulated_home_points=None)
out_data=out_data.assign(Final_accumulated_away_points=None)

In [100]:
for i in range(0,out_data.shape[0]):
    if out_data.iat[i,15]==1:
        out_data.iat[i,16]=out_data.iat[i,5]
    if out_data.iat[i,14]==1:
        out_data.iat[i,17]=out_data.iat[i,6]
    if out_data.iat[i,15]!=1:
        if out_data.iat[i,5]==0 and out_data.iat[i,11]>0:
            out_data.iat[i,16]=out_data.iat[i,5]
        else:
            out_data.iat[i,16]=out_data.iat[i,5]+out_data.iat[i,11]
    if out_data.iat[i,14]!=1:      
        if out_data.iat[i,6]==0 and out_data.iat[i,10]>0:
            out_data.iat[i,17]=out_data.iat[i,6]
        else:
            out_data.iat[i,17]=out_data.iat[i,6]+out_data.iat[i,10]   


In [113]:
table2={"Team_name":team,
      "Points":points}
table2=pd.DataFrame(table2)

for i in range(0,out_data.shape[0]):    
    p=out_data.iloc[i]["Final_accumulated_home_points"]
    t=out_data.iloc[i]["Home_team"]
    for j in range(0,table2.shape[0]):
        if table2.iloc[j]['Team_name'] == t:
            table2.iat[j,1]=table2.iat[j,1]+p       
    p=out_data.iloc[i]["Final_accumulated_away_points"]
    t=out_data.iloc[i]["Away_team"]
    for j in range(0,table2.shape[0]):
        if table2.iloc[j]['Team_name'] == t:
            table2.iat[j,1]=table2.iat[j,1]+p 


In [114]:
table2=table2.sort_values(by=["Points"],ascending=False)
print(table2)
Table=Table.sort_values(by=["Points"],ascending=False)
print(Table)



                    Team_name  Points
6         Raków Częstochowa     71.50
11           Legia Warszawa     66.25
4            Pogoń Szczecin     62.00
3               Lech Poznań     60.25
13            Piast Gliwice     48.50
0             Górnik Zabrze     47.75
8            Zagłębie Lubin     44.25
5            Radomiak Radom     43.00
16             Warta Poznań     42.75
9                  Cracovia     42.00
14              Stal Mielec     38.75
2             Korona Kielce     38.75
17              Widzew Łódź     37.00
1     Jagiellonia Białystok     36.00
7               Wisła Płock     36.00
15            Śląsk Wrocław     34.25
10            Lechia Gdańsk     29.50
12            Miedź Legnica     21.25
                    Team_name  Points
6         Raków Częstochowa        75
11           Legia Warszawa        66
3               Lech Poznań        61
4            Pogoń Szczecin        60
13            Piast Gliwice        53
0             Górnik Zabrze        48
9           

In [103]:
SAVE_FINAL_DATA_FRAME_TO_CSV=False
if SAVE_FINAL_DATA_FRAME_TO_CSV:
    out_data.to_csv("Ekstraklasa_2022-23.csv")

